In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
files = list_files(os.path.join(wdpath, 'Set001'))

In [3]:
files = [file for file in files if '~$' not in file]

In [4]:
set_ID = 'Set001'

In [5]:
start1 = time.time()
frames = []
for year in range(2017, 2023):
    #start2 = time.time()
    tx_dt = read_taxlot(year)
    #end2 = time.time()
    #print("it took {0} minutes to read taxlot in {1}".format(str(round((end2 - start2)/60, 0)), str(year)))
    for file in files:
        #print("processing {0} in {1}".format(file, str(year)))
        wd_dt = read_wd_table(setID = set_ID, file = file)  
        df = wd_dt[wd_dt.IDyear == str(year)].merge(tx_dt[['ORTaxlot', 'geometry']], 
                                                    on='ORTaxlot', 
                                                    how='left')
        frames.append(df)
        #print("processed {0} in {1}".format(file, str(year)))
df = pd.concat(frames, ignore_index=True)      
end1 = time.time()
print("it took {0} minutes to complete {1}".format(str(round((end1 - start1)/60, 0)), set_ID))

it took 16.0 minutes to complete Set001


In [6]:
df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'lots', 'lot', 'cnt_code', 'ORTaxlot', 'year', 'IDyear', 'geometry'],
      dtype='object')

In [7]:
df.reissuance_response_date.dtype

dtype('float64')

In [9]:
df['response_date'] = df['response_date'].dt.strftime("%Y-%m-%d")

In [10]:
df['received_date'] = df['received_date'].dt.strftime("%Y-%m-%d")

In [13]:
df['lots'] = df['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [22]:
df.tail(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,project_id,site_id,record_ID,lots,lot,cnt_code,ORTaxlot,year,IDyear,geometry
2493,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,601,24,2406.00S01.00W3200--000000601,2022,2022,"MULTIPOLYGON (((706600.131 1196247.539, 706585..."
2494,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,700,24,2406.00S01.00W3200--000000700,2022,2022,"MULTIPOLYGON (((706341.268 1196877.808, 706285..."
2495,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,900,24,2406.00S01.00W3200--000000900,2022,2022,"MULTIPOLYGON (((711115.773 1196891.122, 711110..."


In [14]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')

In [17]:
gdf.rename(columns={'wetdet_delin_number': 'wdID', 
                      'address_location_desc':'loc_desc', 
                      'Coord-Source': 'coord_src',
                      'DocumentName':'doc_name',
                      'DecisionLink':'doc_link',
                      'is_batch_file':'isbatfile',
                      'status_name': 'status_nm',
                      'received_date':'receiveddt', 
                      'response_date':'responsedt',
                      'reissuance_response_date':'reissuance', 
                      }, inplace=True)

In [20]:
gdf.to_file(os.path.join(inpath + '\\test', 'matched_records_with_lotID'))

In [24]:
gdf.shape

(2496, 25)

In [23]:
gdf.drop(columns=['lots', 'lot', 'cnt_code'], inplace = True)

In [ ]:
gdf